In [25]:
import numpy as np
from scipy.optimize import least_squares
from datasets import load_dataset
import os

def affine_least_squares(delta1, delta2):
    def error_func(params, x, y):
        a, b = params
        y_pred = a * x + b
        return np.ravel(y - y_pred)  

    initial_params = [1.0, 0.0]  
    result = least_squares(error_func, initial_params, args=(delta1, delta2))

    a, b = result.x
    return a, b

def calculate_absrel(delta2, delta1):
    a, b = affine_least_squares(delta1, delta2)
    pred = delta1 * a + b
    absrel = np.mean(np.abs(delta2 - pred) / delta2)

    return absrel
def calculate_delta(delta2, delta1):
    a, b = affine_least_squares(delta1, delta2)
    pred = delta1 * a + b
    max_ratio = np.maximum(delta2 / pred, pred / delta2)
    delta = max_ratio < 1.25
    num_true_values = np.count_nonzero(delta)
    measures = num_true_values / delta1.size

    return measures

# Carregar os conjuntos de dados
depth_path = "/home/julio/Desktop/depth_npy"
ds = load_dataset("sayakpaul/nyu_depth_v2", split="validation")
all_absrel = []
all_delta = []
# Iterar sobre os dados e calcular as métricas para cada par de deltas
for idx, sample in enumerate(ds):
    # Carregar os deltas
    delta1 = np.load(os.path.join(depth_path, f"{idx + 1}_pred.npy"))
    delta2 = sample["depth_map"]
    
    # Calcular as métricas
    absrel = calculate_absrel(delta2, delta1)
    measures = calculate_delta(delta2, delta1)
    all_absrel.append(absrel)
    all_delta.append(measures)
    mean_absrel = np.mean(all_absrel)
    mean_delta = np.mean(all_delta)
    # Imprimir os resultados
    print(f"Iteração {idx}:")
    print(f"Erro Relativo Absoluto (absrel): {absrel}")
    print(f"Precisão Delta (measures): {measures}")
print(f"Média do Erro Relativo Absoluto: {mean_absrel}")
print(f"Média da Precisão Delta: {mean_delta}")




Iteração 0:
Erro Relativo Absoluto (absrel): 0.030527573078870773
Precisão Delta (measures): 0.993076171875
Iteração 1:
Erro Relativo Absoluto (absrel): 0.030226612463593483
Precisão Delta (measures): 0.9846907552083334
Iteração 2:
Erro Relativo Absoluto (absrel): 0.08736850321292877
Precisão Delta (measures): 0.9664029947916667
Iteração 3:
Erro Relativo Absoluto (absrel): 0.05799563601613045
Precisão Delta (measures): 0.969833984375
Iteração 4:
Erro Relativo Absoluto (absrel): 0.19397999346256256
Precisão Delta (measures): 0.803603515625
Iteração 5:
Erro Relativo Absoluto (absrel): 0.1828605681657791
Precisão Delta (measures): 0.8260807291666666
Iteração 6:
Erro Relativo Absoluto (absrel): 0.12087299674749374
Precisão Delta (measures): 0.89478515625
Iteração 7:
Erro Relativo Absoluto (absrel): 0.12590855360031128
Precisão Delta (measures): 0.9701334635416666
Iteração 8:
Erro Relativo Absoluto (absrel): 0.09718485921621323
Precisão Delta (measures): 0.909931640625
Iteração 9:
Erro Rela